In [2]:
# Import dependencies for transforming MS Docx file to Turtle
from rdflib import Graph, Namespace, Literal, URIRef
from rdflib.namespace import RDF, XSD
import pandas as pd
import re

In [3]:
voc = Graph()
# Load a vocabulary from a ttl file
voc.parse("CedtPocVocab.ttl", format="turtle")
print("CedtPocVocab has %s statements." % len(voc))

# Example content of CedtPocVocab.ttl
# ===================================
# @prefix schema: <http://schema.org/> .
# @prefix skos: <http://www.w3.org/2004/02/skos/core#> .
# @prefix pechar: <https://lustraka.github.io/resources/pechar/> .
#
# pechar:CostsByService a schema:DefinedTermSet ;
#     schema:name "Cost by Service" ;
#     schema:description "ANS cost details required by a reporting table. See ANNEX BII of the Implementing Regulation." ;
#     schema:isPartOf pechar: .

# pechar:201AirTrafficManagement a schema:DefinedTerm ;
#     schema:name "201 Air Traffic Management" ;
#     schema:description "Poskytování letových provozních služeb (ATM) – náklady spojené s vlastním poskytováním řízení letového provozu"@cs ;
#     skos:related ses:AirTrafficManagement ;
#     schema:inDefinedTermSet pechar:CostsByService .
# ===================================

CedtPocVocab has 207 statements.


In [4]:
# Load budget entries from a CSV file
bYYe = pd.read_csv("CedtPocBYYe.csv", sep=",", encoding="utf-8", index_col=0)
print(bYYe.shape)
bYYe.head(3).T

# Example content of CedtPocBYYe.csv
# ===================================
# Index,Service,OrganizationalUnit,CostType,AmountEUR,AmountEnRouteEUR,AmountTerminalEUR,AccountingPeriod
# bYYe001,201 Air Traffic Management,1000 Corporate Services,a Staff Costs,8752200,6867900,1884300,YYYY
# bYYe002,201 Air Traffic Management,1000 Corporate Services,b Operating Costs,624800,585300,39500,YYYY
# ===================================

(125, 7)


Index,bYYe001,bYYe002,bYYe003
Service,201 Air Traffic Management,201 Air Traffic Management,201 Air Traffic Management
OrganizationalUnit,1000 Corporate Services,1000 Corporate Services,1000 Corporate Services
CostType,a Staff Costs,b Operating Costs,c Depreciation Costs
AmountEUR,8752200,624800,18485100
AmountEnRouteEUR,6867900,585300,15084600
AmountTerminalEUR,1884300,39500,3400500
AccountingPeriod,YYYY,YYYY,YYYY


In [5]:
def encode_bYYe_Literal(bYYE):
    """
    Encode the budget entries in the CSV file to RDF triples. Use literals for the values.
    """
    # Create a new graph for the budget entries
    g = Graph()
    
    # Define namespaces
    pechar = Namespace("https://lustraka.github.io/resources/pechar/")
    g.bind("pechar", pechar)

    # Iterate through each row in the DataFrame
    for index, row in bYYE.iterrows():
        # Create a URI for the budget entry
        entry_uri = URIRef(f"https://lustraka.github.io/resources/pechar/{index}")
        
        # Add RDF type for the budget entry
        g.add((entry_uri, RDF.type, pechar.BudgetEntry))
        
        # Add properties to the budget entry
        for col in bYYE.columns:
            g.add((entry_uri, URIRef(f"https://lustraka.github.io/resources/pechar/{col}"), Literal(row[col])))
    
    # Serialize and save the graph to a file
    g.serialize(destination="CedtPocBYYeLit.ttl", format="turtle")
    print("CedtPocBYYeLit.ttl has %s statements." % len(g))
    
    return g

# g = encode_bYYe_Literal(bYYe)


# Example content of CedtPocBYYeLit.ttl
# =====================================
# @prefix pechar: <https://lustraka.github.io/resources/pechar/> .
# @prefix xsd: <http://www.w3.org/2001/XMLSchema#> .

# pechar:bYYe001 a pechar:BudgetEntry ;
#     pechar:AccountingPeriod "YYYY" ;
#     pechar:AmountEUR 8752200 ;
#     pechar:AmountEnRouteEUR 6867900 ;
#     pechar:AmountTerminalEUR 1884300 ;
#     pechar:CostType "a Staff Costs" ;
#     pechar:OrganizationalUnit "1000 Corporate Services" ;
#     pechar:Service "201 Air Traffic Management" .
# =====================================

To enhance semantic alignment with the reference ontology, the algorithm for encoding budget entries into RDF triples was modified to replace selected literal values with corresponding URIs from a controlled vocabulary graph. Specifically, the `encode_bYYe()` function was extended to include a generalized URI resolution mechanism, which searches the vocabulary (`voc`) for resources whose `schema:name` matches the label of interest. For columns such as *Service*, *OrganizationalUnit*, and *CostType*, the function attempts to substitute string literals with URIs found in the vocabulary; if no match is found, the literal is retained as a fallback. This adjustment promotes reuse of existing semantic terms, enhances interoperability, and ensures that the generated RDF graph more faithfully reflects the structure and semantics of the underlying domain ontology.

In [6]:
def encode_bYYe(bYYE, voc):
    """
    Encode the budget entries in the CSV file to RDF triples,
    reusing vocabulary terms from the voc graph.
    """

    g = Graph()
    schema = Namespace("http://schema.org/")
    g.namespace_manager.store.bind("schema", schema, override=True)  # Ensures "schema" is used correctly
    skos = Namespace("http://www.w3.org/2004/02/skos/core#")
    g.bind("skos", skos)
    ses = Namespace("https://eur-lex.europa.eu/eli/reg/2024/2803/oj/")
    g.bind("ses", ses)
    pecha = Namespace("http://data.europa.eu/eli/reg_impl/2019/317#")
    g.bind("pecha", pecha)
    pechar = Namespace("https://lustraka.github.io/resources/pechar/")
    g.bind("pechar", pechar)

    # Helper to lookup URI by label
    def get_uri_by_label(label):
        for s in voc.subjects(predicate=schema.name, object=Literal(label)):
            return s
        return None

    for index, row in bYYE.iterrows():
        entry_uri = URIRef(f"https://lustraka.github.io/resources/pechar/{index}")
        g.add((entry_uri, RDF.type, pechar.BudgetEntry))

        for col in bYYE.columns:
            predicate = URIRef(f"https://lustraka.github.io/resources/pechar/{col}")

            # Special case: replace literal service label with URI from voc
            if col in ["Service", "OrganizationalUnit", "CostType"]:
                term_uri = get_uri_by_label(row[col])
                if term_uri:
                    g.add((entry_uri, predicate, term_uri))
                else:
                    # Fallback to literal if no URI found
                    g.add((entry_uri, predicate, Literal(row[col])))
            else:
                g.add((entry_uri, predicate, Literal(row[col])))

    print("Encoded dataset has %s statements." % len(g))

    # Add pechar:hasTopContributor triples based on AmountEUR
    grouped = bYYE.groupby("Service")["AmountEUR"].idxmax()

    for service_label, top_index in grouped.items():
        service_uri = get_uri_by_label(service_label)
        org_label = bYYE.loc[top_index]["OrganizationalUnit"]
        org_uri = get_uri_by_label(org_label)

        if service_uri and org_uri:
            g.add((service_uri, pechar.hasTopContributor, org_uri))

    print("Encoded dataset with top contributors has %s statements." % len(g))

    # Merge vocabulary graph with the budget entries graph
    for s, p, o in voc:
        g.add((s, p, o))
    
    print("Combined graph has %s statements." % len(g))

    g.serialize(destination="CedtPocBYYe.ttl", format="turtle")
    
    return g

g = encode_bYYe(bYYe, voc)

# Example content of CedtPocBYYe.ttl
# =====================================
# pechar:bYYe001 a pechar:BudgetEntry ;
#     pechar:AccountingPeriod "YYYY" ;
#     pechar:AmountEUR 8752200 ;
#     pechar:AmountEnRouteEUR 6867900 ;
#     pechar:AmountTerminalEUR 1884300 ;
#     pechar:CostType pechar:aStaffCosts ;
#     pechar:OrganizationalUnit pechar:1000_CS ;
#     pechar:Service pechar:201AirTrafficManagement .
#
# pechar:201AirTrafficManagement a schema:DefinedTerm ;
#     schema:description "Poskytování letových provozních služeb (ATM) – náklady spojené s vlastním poskytováním řízení letového provozu"@cs ;
#     schema:inDefinedTermSet pechar:CostsByService ;
#     schema:name "201 Air Traffic Management" ;
#     skos:related ses:AirTrafficManagement ;
#     pechar:hasTopContributor pechar:2111_ACC .
# =====================================

Encoded dataset has 1000 statements.
Encoded dataset with top contributors has 1010 statements.
Combined graph has 1217 statements.


In transforming tabular budgeting data into semantically enriched RDF graphs, a key modeling task is to identify, for each service, the organizational unit with the highest financial contribution. This requires computing the maximum `AmountEUR` value grouped by `Service`, followed by asserting a triple such as `pechar:ServiceX pechar:hasTopContributor OrganizationalUnitY` to support downstream reasoning. Two competing strategies emerge. One approach relies on SPARQL 1.1 subqueries, leveraging `GROUP BY` and `MAX` within the RDF graph to infer top contributors post hoc. While semantically elegant and fully contained within the graph model, this method introduces complexity in both query formulation and result integration when used with Python’s `rdflib`, which provides limited support for modifying graphs based on query outputs. Alternatively, a preprocessing approach externalizes the aggregation logic using `pandas`, computes top contributors prior to RDF serialization, and injects the relevant triples directly into the output graph. Although this method shifts reasoning outside the RDF model, it offers clarity, performance, and precise control over graph construction. In this case, where the input is a TTL file and RDF enrichment is part of a controlled transformation pipeline, the Pandas-based method is preferable: it aligns with the system architecture, enables deterministic augmentation of the graph, and ensures that domain-specific semantics are faithfully encoded without overburdening the SPARQL layer.

In [7]:

def infer_OU_network(g):
    """
    Infers organizational delivery relations based on BudgetEntry and top contributors.
    Adds triples of the form ?OU_a pechar:deliversTo ?OU_b to the graph.
    """

    construct_query = """
    PREFIX schema: <http://schema.org/>
    PREFIX pechar: <https://lustraka.github.io/resources/pechar/>

    CONSTRUCT {
      ?OU_a pechar:deliversTo ?OU_b .
    }
    WHERE {
      ?be a pechar:BudgetEntry ;
          pechar:OrganizationalUnit ?OU_a ;
          pechar:Service ?service .
      ?service pechar:hasTopContributor ?OU_b .
      FILTER (?OU_a != ?OU_b)
    }
    """

    inferred_triples = g.query(construct_query)

    for triple in inferred_triples:
        g.add(triple)

    print("Inferred OU delivery network: added %d triples." % len(inferred_triples))

    g.serialize(destination="CedtPocBYYe.ttl", format="turtle")

    print("Combined graph has %s statements." % len(g))

    return g

g = infer_OU_network(g)


Inferred OU delivery network: added 53 triples.
Combined graph has 1270 statements.


In [ ]:
from rdflib import URIRef, Namespace
from collections import defaultdict, deque

def generate_ou_network_plantuml(g, target_ou, levels):
    """
    Generates PlantUML code for a subgraph of pechar:deliversTo relations 
    leading to the given target organizational unit (OU), up to a given number of levels.
    
    Parameters:
        g         -- an RDFLib Graph
        target_ou -- URIRef of the target OU (e.g., URIRef("https://lustraka.github.io/resources/pechar/2111_ACC"))
        levels    -- integer, number of levels to include
    
    Returns:
        A string with PlantUML diagram code.
    """
    pechar = Namespace("https://lustraka.github.io/resources/pechar/")
    
    # BFS up to the given number of levels
    edges = set()
    visited = set()
    queue = deque([(target_ou, 0)])  # (current node, current level)

    while queue:
        current, level = queue.popleft()
        if level >= levels:
            continue

        for subj in g.subjects(predicate=pechar.deliversTo, object=current):
            if (subj, current) not in edges:
                edges.add((subj, current))
                if subj not in visited:
                    queue.append((subj, level + 1))
                    visited.add(subj)

    # Generate PlantUML code
    uml_lines = ["@startuml", "left to right direction"]
    for s, o in edges:
        s_label = s.split("/")[-1]
        o_label = o.split("/")[-1]
        uml_lines.append(f'"{s_label}" --> "{o_label}"')
    uml_lines.append("@enduml")

        # Save PlantUml code to a file
    with open("CedtPocSubGraph.tex", "w") as f:
        f.write("\n".join(uml_lines))

    return

generate_ou_network_plantuml(g, URIRef("https://lustraka.github.io/resources/pechar/2111_ACC"), 2)

```
@startuml
'skinparam linetype ortho
left to right direction
hide circles
hide empty members
class "2410_AIM"
"4110_HRM" --> "2410_AIM"
"1000_CS" --> "2410_AIM"
"2310_AMaS" --> "2111_ACC"
"4110_HRM" --> "2212_ATS"
"2212_ATS" --> "2410_AIM"
"1000_CS" --> "2212_ATS"
"2410_AIM" --> "2111_ACC"
"4110_HRM" --> "2111_ACC"
"1000_CS" --> "2111_ACC"
"2212_ATS" --> "2111_ACC"
"1000_CS" --> "2310_AMaS"
@enduml
```

![](https://www.plantuml.com/plantuml/png/XOyzIyOm58Nt_8fB5gSNajXPMhheqCMwXvXURJ0V9RaT-kzLX0lI5xgTyEGyXzitSfNs5CL3-mfvSzKbY24Z7Hj2gRGM4V63W0hKyBaIBA6Yfr2oMCE2u4Fr4TlV0zD61oHCxrYRyD6r1j9qMjdXPP92TledFhzE4cwtftz4AwNiE9-1UVoDasEajRR3EDwn6Mtiy3PBmV5oXolyvtA62SlEWBrN5JvEz9YNFSLl)

---

Auxiliery Code

In [8]:
def draft_orgunit_vocab(df):
    """
    Helper function to define the vocabulary for organizational units.
    """
    defined_terms = ""

    # Genereate DefinedTerm instances for each organizational unit
    unique_values = df["OrganizationalUnit"].unique()
    for value in unique_values:
        # Generate a unique URI for each organizational unit
        words = value.split()
        uri = f"pechar:{words[0]}_{''.join(word[0] for word in words[1:])}"
        defined_terms += f"""
        {uri} a schema:DefinedTerm ;
            schema:name "{value}" ;
            schema:description "Organizational unit {value}"@en ;
            schema:inDefinedTermSet pechar:OrgUnits.
        """
    
    # Save the defined terms to a ttl file
    with open("CedtPocOrgUnits.ttl", "w") as f:
        f.write(defined_terms)
    
    return

# draft_orgunit_vocab(bYYe)